<a href="https://colab.research.google.com/github/cwilliamjay/GANS/blob/master/ager_cGANS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone 'https://github.com/PacktPublishing/Generative-Adversarial-Networks-Projects.git'

In [ ]:
%cd Generative-Adversarial-Networks-Projects

In [ ]:
%cd Chapter03

In [ ]:
!pip install -r requirements.txt

In [ ]:
# Before download the dataset move to data directory
%cd data

# Wikipedia : Download faces only
!wget https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar

In [ ]:
# Move to data directory
%cd data

# Extract wiki_crop.tar
!tar -xvf wiki_crop.tar

In [16]:
def load_data(wiki_dir, dataset='wiki'):
    # Load the wiki.mat file
    meta = loadmat(os.path.join(wiki_dir, "{}.mat".format(dataset)))

    # Load the list of all files
    full_path = meta[dataset][0, 0]["full_path"][0]

    # List of Matlab serial date numbers
    dob = meta[dataset][0, 0]["dob"][0]

    # List of years when photo was taken
    photo_taken = meta[dataset][0, 0]["photo_taken"][0]  # year

    # Calculate age for all dobs
    age = [calculate_age(photo_taken[i], dob[i]) for i in range(len(dob))]

    # Create a list of tuples containing a pair of an image path and age
    images = []
    age_list = []
    for index, image_path in enumerate(full_path):
        images.append(image_path[0])
        age_list.append(age[index])

    # Return a list of all images and respective age
    return images, age_list

The photo_taken variable is a list of years and dob is Matlab's serial date number for the corresponding photo in the list. We can calculate the age of the person from the serial date number and the year the photo was taken. Use the following code to calculate the age:

In [15]:
def calculate_age(taken, dob):
    birth = datetime.fromordinal(max(int(dob) - 366, 1))

# assume the photo was taken in the middle of the year
    if birth.month < 7:
       return taken - birth.year
    else:
       return taken - birth.year - 1

In [24]:
import math
import os
import time
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras import Input, Model
from keras.applications import InceptionResNetV2
from keras.callbacks import TensorBoard
from keras.layers import Conv2D, Flatten, Dense, BatchNormalization, Reshape, concatenate, LeakyReLU
from keras.layers import Lambda, Conv2DTranspose, Activation, UpSampling2D, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras_preprocessing import image
from scipy.io import loadmat

In [26]:
input_layer = Input(shape=(64,64,3))

Next, add the first convolution block, which contains a 2D convolution layer with an activation function with the following configurations:
* Filters: 32
* Kernel size: 5
* Strides: 2
* Padding: same
* Activation: LeakyReLU with alpha equal to 0.2:






In [27]:
# 1st Convolutional Block
enc = Conv2D(filters=32, kernel_size=5, strides=2, padding='same')(input_layer)
enc = LeakyReLU(alpha=0.2)(enc)